# Regridding the Roemmich/Gilson ARGO clim onto the ASTE grid

In [ ]:
import xarray as xr
import xmitgcm
import MITgcm_recipes
#import tqdm
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## Get the datasets

### ARGO climatology

In [ ]:
data_dir = '/local/data/artemis/workspace/rdussin/Observations/RG_ARGO_clim/originals/'
ds = xr.open_mfdataset(data_dir + '*.nc', decode_times=False, chunks={'TIME':1})

In [ ]:
ds

### ASTE raw grid

In [ ]:
file_aste_raw = '/local/data/artemis/workspace/rdussin/ASTE/GRID/nc/aste_grid.nc'
aste_raw = xr.open_dataset(file_aste_raw)

### ASTE output grid

In [ ]:
datadir_aste = '/local/data/artemis/workspace/rdussin/ASTE/RUNS/ASTE-Phy-Release1/outputs/'
md = xmitgcm.utils.get_extra_metadata(domain='aste', nx=270)
aste_out = xmitgcm.open_mdsdataset(datadir_aste, iters=[], prefix=['T','S','Eta','U','V'],
                                   geometry='llc', nx=270, extra_metadata=md)

## Vertical interpolation

In [ ]:
# mean
mdsz = MITgcm_recipes.vertical_interpolation(ds, aste_out['Z'], ['ARGO_SALINITY_MEAN',
                                                                 'ARGO_TEMPERATURE_MEAN'],
                                            lonvar='LONGITUDE', 
                                            latvar='LATITUDE',
                                            depth_varin='PRESSURE',
                                            timevar=None)

In [ ]:
# anomalies
adsz = MITgcm_recipes.vertical_interpolation(ds, aste_out['Z'], ['ARGO_SALINITY_ANOMALY',
                                                                 'ARGO_TEMPERATURE_ANOMALY'],
                                             lonvar='LONGITUDE', 
                                             latvar='LATITUDE',
                                             depth_varin='PRESSURE',
                                             timevar='TIME')

In [ ]:
def geo_roll_20_to_180(ds, londim):
    datalon=ds[londim].values
    datalon[datalon>180] -= 360
    ind = (np.abs(datalon-180.)).argmin()
    ds[londim] = datalon
    ds = ds.roll(**{londim: -ind-1})
    return ds

In [ ]:
mdsz180 = geo_roll_20_to_180(mdsz, 'LONGITUDE')

In [ ]:
mdsz180['ARGO_TEMPERATURE_MEAN'].sel(k=0).plot(cmap='gist_ncar')

## Regridding

In [ ]:
adsz180 = geo_roll_20_to_180(adsz, 'LONGITUDE')

In [ ]:
mdsz_aste = MITgcm_recipes.regridding.regrid_2_mitgcm_llc(mdsz180, aste_raw, ['ARGO_SALINITY_MEAN',
                                                                           'ARGO_TEMPERATURE_MEAN'], 
                                                          method='bilinear',
                                                          lonname='LONGITUDE', 
                                                          latname='LATITUDE', 
                                                          point='T', 
                                                          periodic=True, reuse_weights=False,
                                                          regridname='ClimARGO_aste_face')

In [ ]:
adsz_aste = MITgcm_recipes.regridding.regrid_2_mitgcm_llc(adsz180, aste_raw, ['ARGO_SALINITY_ANOMALY',
                                                                           'ARGO_TEMPERATURE_ANOMALY'], 
                                                          method='bilinear',
                                                          lonname='LONGITUDE', 
                                                          latname='LATITUDE', 
                                                          point='T', 
                                                          periodic=True, reuse_weights=True,
                                                          regridname='ClimARGO_aste_face')

## Checking the results

In [ ]:
mdsz_aste['ARGO_TEMPERATURE_MEAN'].isel(face=1, k=0).plot(cmap='gist_ncar')

In [ ]:
mdsz_aste['ARGO_SALINITY_MEAN'].isel(face=1, k=0).plot(cmap='jet')

## Write to netcdf file

In [ ]:
mdsz_aste.to_netcdf(data_dir + '../interp_ASTE/' + 'Roemmich_Clim_ARGO_TS_ASTE.nc')

In [ ]:
adsz_aste = adsz_aste.rename({'TIME': 'time'}, inplace=True).transpose(*('time','face','k','j','i'))

In [ ]:
adsz_aste.to_netcdf(data_dir + '../interp_ASTE/' + 'Roemmich_AnomClim_ARGO_TS_ASTE.nc', unlimited_dims='time')